In [2]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load the data

In [3]:
import datetime
from helpers import *

height, weight, gender = load_data(sub_sample=False, add_outlier=False)
x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

In [4]:
y.shape, tx.shape

((10000,), (10000, 2))

### NB: throughout this laboratory the data has the following format: 
  * there are **N = 10000** data entries
  * **y** represents the column vector containing weight information -- that which we wish to predict/the output (see also the first page of $\texttt{exercise02.pdf}$). Its **shape** is **(N,)**.
  * **tx** represents the matrix $\tilde{X}$ formed by laterally concatenating a column vector of 1s to the column vector of height information -- the input data (see also the first page of $\texttt{exercise02.pdf}$). Its **shape** is **(N,2)**.

# 1. Computing the Cost Function
Fill in the `compute_loss` function below:

In [5]:
def compute_loss(y, tx, w):
    """Calculate the loss using either MSE or MAE.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2,). The vector of model parameters.

    Returns:
        the value of the loss (a scalar), corresponding to the input parameters w.
    """
    e = y - tx.dot(w)
    N = y.shape[0]
    
    # MSE
    #return 1/(2*N) * e.dot(e)

    # MAE
    return 1/(2*N) * np.sum(np.abs(e))
    

# 2. Grid Search

Fill in the function `grid_search()` below:

In [6]:
# from costs import *


def grid_search(y, tx, grid_w0, grid_w1):
    """Algorithm for grid search.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        grid_w0: numpy array of shape=(num_grid_pts_w0, ). A 1D array containing num_grid_pts_w0 values of parameter w0 to be tested in the grid search.
        grid_w1: numpy array of shape=(num_grid_pts_w1, ). A 1D array containing num_grid_pts_w1 values of parameter w1 to be tested in the grid search.

    Returns:
        losses: numpy array of shape=(num_grid_pts_w0, num_grid_pts_w1). A 2D array containing the loss value for each combination of w0 and w1
    """

    losses = np.zeros(((len(grid_w0)), len(grid_w1)))
    
    
    for i in range(len(grid_w0)):
        for j in range(len(grid_w1)):
            losses[i,j] = compute_loss(y, tx, np.array((grid_w0[i], grid_w1[j])))
        
    return losses

Let us play with the grid search demo now!

In [7]:
from grid_search import generate_w, get_best_parameters
from plots import grid_visualization

# Generate the grid of parameters to be swept
grid_w0, grid_w1 = generate_w(num_intervals=50)

# Start the grid search
start_time = datetime.datetime.now()
grid_losses = grid_search(y, tx, grid_w0, grid_w1)

# Select the best combinaison
loss_star, w0_star, w1_star = get_best_parameters(grid_w0, grid_w1, grid_losses)
end_time = datetime.datetime.now()
execution_time = (end_time - start_time).total_seconds()

# Print the results
print(
    "Grid Search: loss*={l}, w0*={w0}, w1*={w1}, execution time={t:.3f} seconds".format(
        l=loss_star, w0=w0_star, w1=w1_star, t=execution_time
    )
)

# Plot the results
fig = grid_visualization(grid_losses, grid_w0, grid_w1, mean_x, std_x, height, weight)
fig.set_size_inches(10.0, 6.0)
fig.savefig("grid_plot")  # Optional saving

Grid Search: loss*=2.4368285589459338, w0*=71.42857142857142, w1*=15.306122448979579, execution time=0.858 seconds


# 3. Gradient Descent

Again, please fill in the functions `compute_gradient` below:

In [8]:
def compute_gradient(y, tx, w):
    """Computes the gradient at w.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2, ). The vector of model parameters.

    Returns:
        An numpy array of shape (2, ) (same shape as w), containing the gradient of the loss at w.
    """
    N = y.shape[0]
    e = y - tx.dot(w)

    grad = -1/N * tx.T.dot(e)

    return grad

In [9]:
compute_gradient(y, tx, np.array((0, -100)))

array([ -73.293922  , -113.47971243])

Please fill in the functions `gradient_descent` below:

In [10]:
def gradient_descent(y, tx, initial_w, max_iters, gamma):
    """The Gradient Descent (GD) algorithm.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize

    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of GD
        ws: a list of length max_iters + 1 containing the model parameters as numpy arrays of shape (2, ),
            for each iteration of GD (as well as the final weights)
    """
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        loss = compute_loss(y, tx, w)
        grad = compute_gradient(y, tx, w)

        w = w - gamma * grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)
        print(
            "GD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
                bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]
            )
        )

    return losses, ws

Test your gradient descent function through gradient descent demo shown below:

In [11]:
# from gradient_descent import *
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.2

# Initialization
w_initial = np.array([0, 0])

# Start gradient descent.
start_time = datetime.datetime.now()
gd_losses, gd_ws = gradient_descent(y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("GD: execution time={t:.3f} seconds".format(t=exection_time))

GD iter. 0/49: loss=36.64696100105259, w0=14.65878440042104, w1=2.695942486997799
GD iter. 1/49: loss=29.31756880084207, w0=26.385811920757867, w1=4.852696476596035
GD iter. 2/49: loss=23.454055040673655, w0=35.76743393702733, w1=6.578099668274626
GD iter. 3/49: loss=18.76324403253893, w0=43.2727315500429, w1=7.958422221617504
GD iter. 4/49: loss=15.01059522603114, w0=49.276969640455356, w1=9.062680264291808
GD iter. 5/49: loss=12.008559081647606, w0=54.08036011278532, w1=9.946086698431252
GD iter. 6/49: loss=9.607998994947218, w0=57.9230724906493, w1=10.652811845742807
GD iter. 7/49: loss=7.691746946347693, w0=60.99724239294048, w1=11.218191963592052
GD iter. 8/49: loss=6.1791761277345945, w0=63.456578314773424, w1=11.670496057871448
GD iter. 9/49: loss=5.012483231293477, w0=65.42404705223977, w1=12.032339333294965
GD iter. 10/49: loss=4.145084742327512, w0=66.99802204221285, w1=12.32181395363378
GD iter. 11/49: loss=3.5184724520407125, w0=68.25720203419132, w1=12.553393649904832
GD i

In [12]:
# Time Visualization
from ipywidgets import IntSlider, interact


def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        gd_losses,
        gd_ws,
        grid_losses,
        grid_w0,
        grid_w1,
        mean_x,
        std_x,
        height,
        weight,
        n_iter,
    )
    fig.set_size_inches(10.0, 6.0)
    return fig


interact(plot_figure, n_iter=IntSlider(min=1, max=len(gd_ws)))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>

# 4. Stochastic gradient descent

In [13]:
def compute_stoch_gradient(y, tx, w):
    """Compute a stochastic gradient at w from a data sample batch of size B, where B < N, and their corresponding labels.

    Args:
        y: numpy array of shape=(B, )
        tx: numpy array of shape=(B,2)
        w: numpy array of shape=(2, ). The vector of model parameters.

    Returns:
        A numpy array of shape (2, ) (same shape as w), containing the stochastic gradient of the loss at w.
    """

    return compute_gradient(y,tx,w)


def stochastic_gradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    """The Stochastic Gradient Descent algorithm (SGD).

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        batch_size: a scalar denoting the number of data points in a mini-batch used for computing the stochastic gradient
        max_iters: a scalar denoting the total number of iterations of SGD
        gamma: a scalar denoting the stepsize

    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SGD
    """

    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w

    batch = batch_iter(y, tx, batch_size, num_batches=max_iters)
    for n_iter in range(max_iters):
        minibatch_y, minibatch_tx = next(batch)
        grad = compute_stoch_gradient(minibatch_y, minibatch_tx, w)
        loss = compute_loss(y, tx, w)

        w = w - gamma * grad
        
        # store w and loss
        ws.append(w)
        losses.append(loss)

        print(
            "SGD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
                bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]
            )
        )
    return losses, ws

In [14]:
# from stochastic_gradient_descent import *

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.1 #0.1
batch_size = 1

# Initialization
w_initial = np.array([0, 0])

# Start SGD.
start_time = datetime.datetime.now()
sgd_losses, sgd_ws = stochastic_gradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma
)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SGD: execution time={t:.3f} seconds".format(t=exection_time))

SGD iter. 0/49: loss=36.64696100105259, w0=9.08546438712487, w1=8.331506501947777
SGD iter. 1/49: loss=32.10422880749016, w0=14.655335433454551, w1=5.34725870795394
SGD iter. 2/49: loss=29.319293284325315, w0=20.698553296865914, w1=7.975923905428554
SGD iter. 3/49: loss=26.29768435261964, w0=24.80001204018882, w1=3.773142520513775
SGD iter. 4/49: loss=24.246954980958176, w0=30.437807550854075, w1=10.505789053738276
SGD iter. 5/49: loss=21.428057225625555, w0=35.85369607953368, w1=13.775609995430793
SGD iter. 6/49: loss=18.72011296128576, w0=39.18808012711484, w1=11.967762514776222
SGD iter. 7/49: loss=17.05292093749518, w0=43.02914335308831, w1=12.701524057241471
SGD iter. 8/49: loss=15.132389324508443, w0=46.0838843039948, w1=18.78906528372661
SGD iter. 9/49: loss=13.605324219131278, w0=47.805119631911445, w1=20.69505858416315
SGD iter. 10/49: loss=12.748461137497658, w0=51.69159595774747, w1=15.803325455652137
SGD iter. 11/49: loss=10.80129387437303, w0=53.206767590214426, w1=15.0824

In [15]:
# Time Visualization
from ipywidgets import IntSlider, interact


def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        sgd_losses,
        sgd_ws,
        grid_losses,
        grid_w0,
        grid_w1,
        mean_x,
        std_x,
        height,
        weight,
        n_iter,
    )
    fig.set_size_inches(10.0, 6.0)
    return fig

interact(plot_figure, n_iter=IntSlider(min=1, max=max_iters))

interactive(children=(IntSlider(value=1, description='n_iter', max=50, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>

# 5. Effect of Outliers and MAE Cost Function

In [16]:
import datetime
from helpers import *

height, weight, gender = load_data(sub_sample=True, add_outlier=True)

x, mean_x, std_x = standardize(height)
y, tx = build_model_data(x, weight)

In [17]:
y.shape, tx.shape

((202,), (202, 2))

In [18]:
from plots import gradient_descent_visualization

# Define the parameters of the algorithm.
max_iters = 50
gamma = 0.7

# Initialization
w_initial = np.array([0, 0])

# Start gradient descent.
start_time = datetime.datetime.now()

sgd_losses, sgd_ws = stochastic_gradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma
)
# ***************************************************
# INSERT YOUR CODE HERE
# TODO: fit the model to the subsampled data / subsampled data with outliers and visualize the cloud of points
#       and the model fit
# ***************************************************
#raise NotImplementedError


end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("GD: execution time={t:.3f} seconds".format(t=exection_time))

SGD iter. 0/49: loss=37.03390292746319, w0=42.51564598253335, w1=-41.20051533452664
SGD iter. 1/49: loss=24.753802005529337, w0=-1.7703516464227462, w1=33.70622557879146
SGD iter. 2/49: loss=37.91907875067454, w0=52.25956970302083, w1=31.856407807885223
SGD iter. 3/49: loss=11.563973520632196, w0=68.80079109860657, w1=23.654703452167276
SGD iter. 4/49: loss=4.483326123517069, w0=75.11873474062834, w1=23.892733338364916
SGD iter. 5/49: loss=4.3495101977132755, w0=76.30099471638124, w1=23.16478093593615
SGD iter. 6/49: loss=4.31582480659576, w0=75.89031720373347, w1=23.47393722839585
SGD iter. 7/49: loss=4.332263972415219, w0=71.36984949232968, w1=23.995820740654764
SGD iter. 8/49: loss=4.265517971532613, w0=77.05950295028508, w1=18.65664206973942
SGD iter. 9/49: loss=3.5378833942023924, w0=72.46684174263747, w1=21.43349248073915
SGD iter. 10/49: loss=3.4762218864354786, w0=68.59231957583427, w1=15.829614219903673
SGD iter. 11/49: loss=3.292337150923489, w0=77.28693771371267, w1=18.26838

In [20]:
# Time Visualization
from ipywidgets import IntSlider, interact


def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        gd_losses,
        gd_ws,
        grid_losses,
        grid_w0,
        grid_w1,
        mean_x,
        std_x,
        height,
        weight,
        n_iter,
    )
    fig.set_size_inches(10.0, 6.0)
    return fig


interact(plot_figure, n_iter=IntSlider(min=1, max=len(gd_ws)))

interactive(children=(IntSlider(value=1, description='n_iter', max=51, min=1), Output()), _dom_classes=('widge…

<function __main__.plot_figure(n_iter)>

# 6. Subgradient descent

In [21]:
def compute_subgradient_mae(y, tx, w):
    """Compute a subgradient of the MAE at w.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        w: numpy array of shape=(2, ). The vector of model parameters.

    Returns:
        A numpy array of shape (2, ) (same shape as w), containing the subgradient of the MAE at w.
    """
    
    N = y.shape[0]
    e = y - tx.dot(w)

    t_mod = tx.copy()
    
    t_mod[e < 0] = -tx[e < 0]
    t_mod[e == 0] = 0

    if not e.any():
        print("Error 0 encountered")
    
    grad = 1/N * np.sum(-t_mod, axis=0)

    return grad

In [22]:
compute_subgradient_mae(y, tx, (-150,220))

array([-0.73267327,  0.40102423])

In [23]:
def subgradient_descent(y, tx, initial_w, max_iters, gamma):
    """The SubGradient Descent (SubGD) algorithm.

    Args:
        y: numpy array of shape=(N, )
        tx: numpy array of shape=(N,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        max_iters: a scalar denoting the total number of iterations of GD
        gamma: a scalar denoting the stepsize

    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SubGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SubGD
    """
    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w
    for n_iter in range(max_iters):
        
        loss = compute_loss(y, tx, w)
        grad = compute_subgradient_mae(y, tx, w)

        w = w - gamma * grad

        ws.append(w)
        losses.append(loss)
        print(
            "SubGD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
                bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]
            )
        )

    return losses, ws

In [24]:
# Define the parameters of the algorithm.
max_iters = 5000
gamma = 10#0.7
batch_size = 1

# Initialization
w_initial = np.array([0, 0])

# Start SubSGD.
start_time = datetime.datetime.now()
subgd_losses, subgd_ws = subgradient_descent(y, tx, w_initial, max_iters, gamma)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SubGD: execution time={t:.3f} seconds".format(t=exection_time))

SubGD iter. 0/4999: loss=37.03390292746319, w0=10.0, w1=8.72789189655816e-15
SubGD iter. 1/4999: loss=32.033902927463195, w0=20.0, w1=1.745578379311632e-14
SubGD iter. 2/4999: loss=27.033902927463192, w0=30.0, w1=2.6183675689674482e-14
SubGD iter. 3/4999: loss=22.03390292746319, w0=40.0, w1=3.491156758623264e-14
SubGD iter. 4/4999: loss=17.033902927463192, w0=50.0, w1=4.363945948279081e-14
SubGD iter. 5/4999: loss=12.093040931680612, w0=59.40594059405941, w1=0.8868917349265609
SubGD iter. 6/4999: loss=8.286950184355888, w0=64.95049504950495, w1=5.294136958480561
SubGD iter. 7/4999: loss=5.90732168242875, w0=68.91089108910892, w1=10.409491987173137
SubGD iter. 8/4999: loss=3.9148833810078516, w0=71.88118811881189, w1=15.166515724701
SubGD iter. 9/4999: loss=2.7038883834695415, w0=73.16831683168319, w1=16.168245336722173
SubGD iter. 10/4999: loss=2.666056173044877, w0=72.67326732673268, w1=15.905565298680443
SubGD iter. 11/4999: loss=2.655460125516539, w0=72.67326732673268, w1=15.9620261

In [25]:
from ipywidgets import IntSlider, interact


def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        subgd_losses,
        subgd_ws,
        grid_losses,
        grid_w0,
        grid_w1,
        mean_x,
        std_x,
        height,
        weight,
        n_iter,
    )
    fig.set_size_inches(10.0, 6.0)
    return fig


interact(plot_figure, n_iter=IntSlider(min=1, max=len(subgd_ws)))

interactive(children=(IntSlider(value=1, description='n_iter', max=5001, min=1), Output()), _dom_classes=('wid…

<function __main__.plot_figure(n_iter)>

# Stochastic Subgradient Descent

**NB** for the computation of the subgradient you can reuse the `compute_subgradient` method that you implemented above, just making sure that you pass in a minibatch as opposed to the full data.

In [26]:
def stochastic_subgradient_descent(y, tx, initial_w, batch_size, max_iters, gamma):
    """Compute a stochastic subgradient at w from a data sample batch of size B, where B < N, and their corresponding labels.

    Args:
        y: numpy array of shape=(B, )
        tx: numpy array of shape=(B,2)
        initial_w: numpy array of shape=(2, ). The initial guess (or the initialization) for the model parameters
        batch_size: a scalar denoting the number of data points in a mini-batch used for computing the stochastic subgradient
        max_iters: a scalar denoting the total number of iterations of SubSGD
        gamma: a scalar denoting the stepsize

    Returns:
        losses: a list of length max_iters containing the loss value (scalar) for each iteration of SubSGD
        ws: a list of length max_iters containing the model parameters as numpy arrays of shape (2, ), for each iteration of SubSGD
    """

    # Define parameters to store w and loss
    ws = [initial_w]
    losses = []
    w = initial_w

    batch = batch_iter(y, tx, batch_size, num_batches=max_iters)

    for n_iter in range(max_iters):
        minibatch_y, minibatch_tx = next(batch)
        
        loss = compute_loss(y, tx, w)
        grad = compute_subgradient_mae(minibatch_y, minibatch_tx, w)

        w = w - gamma * grad

        ws.append(w)
        losses.append(loss)

        print(
            "SubSGD iter. {bi}/{ti}: loss={l}, w0={w0}, w1={w1}".format(
                bi=n_iter, ti=max_iters - 1, l=loss, w0=w[0], w1=w[1]
            )
        )
    return losses, ws

In [27]:
# Define the parameters of the algorithm.
max_iters = 500
gamma = 0.4
batch_size = 1

# Initialization
w_initial = np.array([0, 0])

# Start SubSGD.
start_time = datetime.datetime.now()
subsgd_losses, subsgd_ws = stochastic_subgradient_descent(
    y, tx, w_initial, batch_size, max_iters, gamma
)
end_time = datetime.datetime.now()

# Print result
exection_time = (end_time - start_time).total_seconds()
print("SubSGD: execution time={t:.3f} seconds".format(t=exection_time))

SubSGD iter. 0/499: loss=37.03390292746319, w0=0.4, w1=0.2666183576951858
SubSGD iter. 1/499: loss=36.833902927463186, w0=0.8, w1=0.12717445786138531
SubSGD iter. 2/499: loss=36.63390292746319, w0=1.2000000000000002, w1=-0.12436568768545542
SubSGD iter. 3/499: loss=36.43390292746319, w0=1.6, w1=0.47929215881676757
SubSGD iter. 4/499: loss=36.23390292746319, w0=2.0, w1=0.7788239025034638
SubSGD iter. 5/499: loss=36.03390292746319, w0=2.4, w1=0.62369236994199
SubSGD iter. 6/499: loss=35.83390292746319, w0=2.8, w1=0.4314849655556604
SubSGD iter. 7/499: loss=35.63390292746319, w0=3.1999999999999997, w1=0.8040889069513492
SubSGD iter. 8/499: loss=35.43390292746319, w0=3.5999999999999996, w1=0.5581851421166428
SubSGD iter. 9/499: loss=35.23390292746319, w0=3.9999999999999996, w1=0.44113741647365823
SubSGD iter. 10/499: loss=35.03390292746319, w0=4.3999999999999995, w1=0.21287583225275608
SubSGD iter. 11/499: loss=34.83390292746319, w0=4.8, w1=-0.25492895049331715
SubSGD iter. 12/499: loss=34

In [28]:
from ipywidgets import IntSlider, interact


def plot_figure(n_iter):
    fig = gradient_descent_visualization(
        subsgd_losses,
        subsgd_ws,
        grid_losses,
        grid_w0,
        grid_w1,
        mean_x,
        std_x,
        height,
        weight,
        n_iter,
    )
    fig.set_size_inches(10.0, 6.0)
    return fig


interact(plot_figure, n_iter=IntSlider(min=1, max=len(subsgd_ws)))

interactive(children=(IntSlider(value=1, description='n_iter', max=501, min=1), Output()), _dom_classes=('widg…

<function __main__.plot_figure(n_iter)>